In [1]:
# # for colab compatibility
# !pip install shap
# !pip install ppscore
# !pip install xgboost
# !pip install lightgbm

In [2]:
# %run data_load_wrapper.ipynb

In [3]:
from tqdm.notebook import trange
import sklearn.datasets as data
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn import *
from sklearn.feature_selection import *
from sklearn.datasets import make_regression
from sklearn.utils import Bunch
import matplotlib as plt
import seaborn as sns
import shap
import numpy as np
import ppscore as pps
from datetime import datetime
import xgboost

In [16]:
from heapq import nlargest

def sklearn_to_df(dataset, with_target=True):
    df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
    y = dataset.target
    if with_target == True:
        df = df.join(pd.DataFrame(y, columns=['target']), on=df.index)
        return df 
    else:
        return df, y

def get_feat_imp_data_based(imp_type, x_train, y_train):
    imp_type_dict = {
        'mutual_info' : 'mutual_info_regression(x_train, y_train)',
        'f_test' : 'f_regression(x_train, y_train)[0]',       
        'pps' : "pps.predictors(x_train.join(y_train), 'target', sorted=False, output='df').model_score",
        'pearson' : "abs(x_train.join(y_train).corr(method='pearson').target).drop('target', axis=0)",
        'spearman': "abs(x_train.join(y_train).corr(method='spearman').target).drop('target', axis=0)",
        'kendall' : "abs(x_train.join(y_train).corr(method='kendall').target).drop('target', axis=0)",
    
    }
    timings = %timeit -o -n1 -r10 eval(imp_type_dict.get(imp_type))
    timings = np.array(timings.timings)
    return eval(imp_type_dict.get(imp_type)), timings

def get_feat_imp_model_based(imp_type, m, x_train, y_train):
    imp_type_dict = {
        'gini' : "m.feature_importances_",
        'pimp' : f"""permutation_importance(m, x_train, y_train, n_repeats=5, 
        random_state=0).importances_mean""",
        'shap' : "abs(np.array(shap.TreeExplainer(m).shap_values(x_train))).sum(axis=0)"
    }
    timings = %timeit -o -n1 -r10 eval(imp_type_dict.get(imp_type))
    timings = np.array(timings.timings)
    return eval(imp_type_dict.get(imp_type)), timings

def get_syn_stats_data_based(imp_types, datasets, dataset_states, 
                             informative_pct):
    results = []
    datasets_str = list(datasets.keys())
    for j in trange(len(datasets_str), desc=f'Dataset progress'):
        for i in trange(len(dataset_states), desc=f'Processing {datasets_str[j]}'):
            for k in trange(len(imp_types)):
                imp_type = imp_types[k] 
                dataset_state = dataset_states[i]
                dataset_str = datasets_str[j]
                dataset = datasets.get(datasets_str[j])
                df = sklearn_to_df(dataset)
                x_train, x_test, y_train, y_test = model_selection.train_test_split(df,
                                                                   dataset.target, 
                                                                   test_size=0.3, 
                                                                   random_state=dataset_state
                                                                  )
                y_train = x_train.target
                x_train = x_train.drop('target', axis=1)
                feat_size = len(x_train.columns)
                sel_feat_size = int(feat_size * informative_pct)
                fi, timings = get_feat_imp_data_based(imp_type, x_train, y_train)
                fi_order = nlargest(feat_size, range(len(fi)), key=lambda idx: fi[idx])
                fi_selected = fi_order[:sel_feat_size]
                result = [dataset_str, imp_type, 0, dataset_state, fi_order, fi_selected, timings]
#                 print(result)
                results.append(result)
#                 print(results)
        columns = ['dataset', 'imp_type', 'random_state', 'data_split', 
                       'fi_order', 'fi_selected', 'timings']
        res = pd.DataFrame(results, columns=columns)
    return res
        
def get_syn_stats_model_based(imp_types, datasets, 
                              dataset_states, informative_pct):
    results = []
    datasets_str = list(datasets.keys())
    for j in trange(len(datasets_str), desc=f'Dataset progress'):
        for i in trange(len(dataset_states), desc=f'Processing {datasets_str[j]}'):
            for k in trange(len(imp_types)):
                imp_type = imp_types[k] 
                dataset_state = dataset_states[i]
                dataset_str = datasets_str[j]
                dataset = datasets.get(datasets_str[j])
                df = sklearn_to_df(dataset, with_target=True)
                x_train, x_test, y_train, y_test = model_selection.train_test_split(df,
                                                                   dataset.target, 
                                                                   test_size=0.3, 
                                                                   random_state=dataset_state
                                                                  )
                y_train = x_train.target
                x_train = x_train.drop('target', axis=1)
                feat_size = len(x_train.columns)
                sel_feat_size = int(feat_size * informative_pct)
                m = ensemble.RandomForestRegressor(random_state=0, n_jobs=-1)
                m.fit(x_train, y_train)
                fi, timings = get_feat_imp_model_based(imp_type, m, x_train, y_train)
                fi_order = nlargest(feat_size, range(len(fi)), key=lambda idx: fi[idx])
                fi_selected = fi_order[:sel_feat_size]
                result = [dataset_str, imp_type, 0, dataset_state, fi_order, fi_selected, timings]
#                 print(result)
                results.append(result)
#                 print(results)
        columns = ['dataset', 'imp_type', 'random_state', 'data_split', 
                       'fi_order', 'fi_selected', 'timings']
        res = pd.DataFrame(results, columns=columns)
    return res


from wrapt_timeout_decorator import *

@timeout(60)
def get_shap_fi(imp_type, dataset_state, dataset_str, dataset, informative_pct):
    
    result = []
    df = sklearn_to_df(dataset, with_target=True)
    x_train, x_test, y_train, y_test = model_selection.train_test_split(df,
                                                       dataset.target, 
                                                       test_size=0.3, 
                                                       random_state=dataset_state
                                                      )
    y_train = x_train.target
    x_train = x_train.drop('target', axis=1)
    feat_size = len(x_train.columns)
    sel_feat_size = int(feat_size * informative_pct)
    m = ensemble.RandomForestRegressor(random_state=0, n_jobs=-1)
    m.fit(x_train, y_train)
    fi, timings = get_feat_imp_model_based(imp_type, m, x_train, y_train)
    print(fi.shape)
#     fi = fi.mean(axis=0)
#     print(fi.shape)
    #                 imp_type, model, x_train, y_train, random_state
    fi_order = nlargest(feat_size, range(len(fi)), key=lambda idx: fi[idx])
    fi_selected = fi_order[:sel_feat_size]
    result = [dataset_str, imp_type, 0, dataset_state, fi_order, fi_selected, timings]
    #                 print(result)
#     result.append(result)
    return result

def get_syn_stats_shap_based(imp_types, datasets, 
                              dataset_states, informative_pct):
    results = []
    datasets_str = list(datasets.keys())
    for j in trange(len(datasets_str), desc=f'Dataset progress'):
        try:
            for i in trange(len(dataset_states), desc=f'Processing {datasets_str[j]}'):
                for k in trange(len(imp_types)):
                    imp_type = imp_types[k] 
                    dataset_state = dataset_states[i]
                    dataset_str = datasets_str[j]
                    dataset = datasets.get(datasets_str[j])
                    result = get_shap_fi(imp_type, dataset_state, dataset_str, dataset, informative_pct)
                    results.append(result)
        except TimeoutError as e:
            print(f'skipping the execution of {dataset_str} because of {e}')
            
    columns = ['dataset', 'imp_type', 'random_state', 'data_split', 
                       'fi_order', 'fi_selected', 'timings']
    result_df = pd.DataFrame(results, columns=columns)
    return result_df

def coerce_synthethic_datasets(samples, features, percent_informative, random_state):
    data, target = make_regression(n_samples=samples, n_features=features, 
                                       n_informative=int(percent_informative*features),
                                       shuffle=False,
                                       random_state=random_state
                                      )
    feature_names = [f'x_{i}' for i in range(features)]
    # data, target, feature_names
    data_bunch = Bunch(
        data=data,
        target=target,
        feature_names=feature_names,
        informative_features=feature_names[:int(percent_informative*features)]
                        )
    return data_bunch

In [5]:
imp_types_data = [
             'mutual_info',
             'f_test', 
             'pearson', 
             'spearman', 
             'kendall', 
             'pps'
            ]
imp_types_model = [
             'gini',
             'pimp', 
            ]

n_jobs = 3

random_states = [i for i in range(1)]
dataset_states = [i for i in range(10)]

features = [10, 100]
samples = [100, 1000, 10000]
# features = [100]
# samples = [10000]
# features = [10]
# samples = [100]
synthethic_states = [i for i in range(1)]
percent_informative = 0.2
tuples = []
for i in features:
    for j in samples:
        if i != j:
            for k in synthethic_states:            
                tuples.append((i,j,k))
datasets = {}

for feat, samples, random_seed in tuples:
    datasets[f'regr_features_{feat}_samples_{samples}_seed_{random_seed}'] = coerce_synthethic_datasets(samples, feat, 0.2, random_seed)

In [6]:
res_data = get_syn_stats_data_based(imp_types_data, datasets, dataset_states, 0.2)

Dataset progress:   0%|          | 0/5 [00:00<?, ?it/s]

Processing classif_features_10_samples_100_seed_0:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

12.4 ms ± 1.3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.55 ms ± 145 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
2.06 ms ± 180 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
2.28 ms ± 430 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
15.1 ms ± 405 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
87 ms ± 5.12 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/6 [00:00<?, ?it/s]

...

3.16 s ± 6.29 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.7 ms ± 2.63 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
126 ms ± 727 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
300 ms ± 4.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
7.24 s ± 46.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.25 s ± 3.74 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/6 [00:00<?, ?it/s]

3.18 s ± 4.59 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.05 ms ± 1.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
125 ms ± 749 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
293 ms ± 4.09 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
7.25 s ± 4.85 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.18 s ± 2.88 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/6 [00:00<?, ?it/s]

3.15 s ± 6.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
3.49 ms ± 526 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
125 ms ± 699 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
298 ms ± 4.17 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
7.23 s ± 37.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.14 s ± 7.9 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [12]:
res_model = get_syn_stats_model_based(imp_types_model, datasets, dataset_states, 0.2)

Dataset progress:   0%|          | 0/5 [00:00<?, ?it/s]

Processing classif_features_10_samples_100_seed_0:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

15.3 ms ± 2.74 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.01 s ± 20.9 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/2 [00:00<?, ?it/s]

...

18.8 ms ± 1.07 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
31.1 s ± 907 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/2 [00:00<?, ?it/s]

17.1 ms ± 633 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
30.4 s ± 847 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/2 [00:00<?, ?it/s]

17.9 ms ± 699 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
32 s ± 1.41 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [17]:
# shap is slow, need timeout sig to stop process
res_shap = get_syn_stats_shap_based(['shap'], datasets, dataset_states, 0.2)

Dataset progress:   0%|          | 0/5 [00:00<?, ?it/s]

Processing classif_features_10_samples_100_seed_0:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

47.2 ms ± 1.58 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
(10,)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

skipping the execution of classif_features_100_samples_1000_seed_0 because of Function get_shap_fi timed out after 60.0 seconds


Processing classif_features_100_samples_10000_seed_0:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

skipping the execution of classif_features_100_samples_10000_seed_0 because of Function get_shap_fi timed out after 60.0 seconds


In [18]:
frames = [res_data, res_model, res_shap]

In [19]:
final_df = pd.concat(frames);

In [21]:
final_df;

In [22]:
# export results to local dir as csv file
final_df.to_csv(f'regression_synthethic_data_{datetime.today().strftime("%Y-%b-%d-%H:%M:%S")}.csv',
             index=False)

In [23]:
final_df

,dataset,imp_type,random_state,data_split,fi_order,fi_selected,timings
0,classif_features_10_samples_100_seed_0,mutual_info,0,0,"[0, 1, 6, 3, 8, 7, 4, 2, 5, 9]","[0, 1]","[0.013693295000848593, 0.01221649599756347, 0...."
1,classif_features_10_samples_100_seed_0,f_test,0,0,"[0, 1, 6, 9, 8, 4, 2, 3, 5, 7]","[0, 1]","[0.00182100299934973, 0.0017979320000449661, 0..."
2,classif_features_10_samples_100_seed_0,pearson,0,0,"[0, 1, 6, 9, 8, 4, 2, 3, 5, 7]","[0, 1]","[0.002302085998962866, 0.0017508220007584896, ..."
3,classif_features_10_samples_100_seed_0,spearman,0,0,"[0, 1, 6, 8, 4, 9, 2, 3, 7, 5]","[0, 1]","[0.0024746769995545037, 0.0033698229999572504,..."
4,classif_features_10_samples_100_seed_0,kendall,0,0,"[0, 1, 6, 8, 4, 9, 2, 3, 7, 5]","[0, 1]","[0.015187065000645816, 0.01486127299722284, 0...."
...,...,...,...,...,...,...,...
15,classif_features_10_samples_1000_seed_0,shap,0,5,"[1, 0, 6, 3, 9, 8, 5, 2, 7, 4]","[1, 0]","[3.669437716001994, 3.6497950710036093, 3.6544..."
16,classif_features_10_samples_1000_seed_0,shap,0,6,"[1, 0, 5, 2, 7, 8, 3, 6, 9, 4]","[1, 0]","[3.6540138150012353, 3.6743375939986436, 3.647..."
17,classif_features_10_samples_1000_seed_0,shap,0,7,"[1, 0, 5, 7, 3, 6, 8, 9, 2, 4]","[1, 0]","[4.042630871001165, 3.899649742001202, 3.97792..."
18,classif_features_10_samples_1000_seed_0,shap,0,8,"[1, 0, 5, 7, 9, 4, 3, 8, 6, 2]","[1, 0]","[3.685066628997447, 3.6729397809976945, 3.6540..."


In [ ]:
## plots
# import seaborn as sns
# plot = sns.lineplot(x='iteration', y='mcc_test', hue='imp_type',
#             data=df_res.query('''model == "XGBClassifier"''')
#             )

# plot = sns.lineplot(x='iteration', y='mcc_test', hue='imp_type',
#             data=df_res.query('''model == "DecisionTreeClassifier"''')
#             )

# plot = sns.lineplot(x='iteration', y='mcc_test', hue='imp_type',
#             data=df_res.query('''model == "RandomForestClassifier"''')
#             )

In [ ]:
# export results to local dir as csv file
# df_res.to_csv(f'classification_synthethic_data_{datetime.today().strftime("%Y-%b-%d-%H:%M:%S")}.csv',
#              index=False)